In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/MIB_lab/mydataset.py .

In [ ]:
## Install Required Packages ##
# !pip istall --user torchvision

import numpy as np
import pandas as pd
import os

from mydataset import myDataset

## PyTorch ##
import torch
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## TorchVision ##
import torchvision
import torchvision.transforms as transforms
from torchvision.io import read_image

## Utils ##
# from utils.ImageDataset import ImageDataset
# from utils.EarlyStopping import EarlyStopping

## Plot ##
import matplotlib.pyplot as plt


## Seed ##
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

## CUDNN ##
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True

In [ ]:
## FOR TESTING: Fraction of Training dataset to use ##
train_fraction = 0.01

## Images per batch ##
batch_size= 7

## Image Directory ##
image_directory = '/content/drive/MyDrive/MIB_lab/images/train'

# Metadata array:
metadata_path= '/content/drive/MyDrive/MIB_lab/images/train/train.csv'

## Metadata dataframe ##
metadata = pd.read_csv( metadata_path, header=None )

In [ ]:
class myDataset(Dataset):
  def __init__(self,metadata,img_dir,transform=None):
    #check if image_dir exist
    if (not os.path.isdir(img_dir)):
      exit('Directory does not exist.')
    
    self.img_labels = metadata
    self.img_dir = img_dir
    self.transform = transform

  def __len__(self):
    return len(self.img_labels)
  
  def __getitem__(self,idx):
    img_path = os.path.join(self.img_dir,str(self.img_labels.iloc[idx,0])+'.jpg')
    _image = read_image(img_path)
    label = self.img_labels.iloc[idx,1]

    if _image.dtype == torch.uint8:
      _image = _image/255.
      
    if self.transform:
      _image = self.transform(_image)
    
    #return _image,label 
    return {'image':_image,'label':label}

In [ ]:
## TESTING: Shuffle and Subset ##
test_df= metadata.sample(frac=train_fraction).reset_index(drop=True)
## Testing Dataset: Define ##
test_dataset = myDataset(metadata=test_df, img_dir=image_directory, transform=None) 

## Testing Dataset: Dataloader ##
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, 
    shuffle=False, 
    # num_workers=40
)

In [ ]:

## Convolutional Encoder ## 
encoder = (
            ## Conv, ReLu ##
            nn.Conv2d(in_channels=3,out_channels=6,kernel_size=3,padding = 1), # Channels (3 -> 32) 
            nn.ReLU(),
            ## MaxPool ##
            nn.MaxPool2d( stride=2, kernel_size=2, padding=0), # Spatial (256 -> 128)
            ## Conv, ReLu ##
            nn.Conv2d(in_channels=6,out_channels=12,kernel_size=3,padding = 1), # Channels (64 -> 32)
            nn.ReLU(),
            ## MaxPool ##
            nn.MaxPool2d( stride=2, kernel_size=2, padding=0),
            ## Conv, ReLu ##
            nn.Conv2d(in_channels=12,out_channels=24,kernel_size=3,padding = 1), # Channels (64 -> 32)
            nn.ReLU(),
            ## MaxPool ##
            nn.MaxPool2d( stride=2, kernel_size=2, padding=0),
)

In [ ]:
## First Image Batch ## 
first_batch= iter( test_loader ).next()['image']
print('First Batch')
print( '%s'% (first_batch.shape,) )
print()

print('### ENCODE ###')
## Encoder: Iterate across Layer ##
x = first_batch
for layer in encoder:
    print(layer)
    x = layer(x)
    if( type(layer) == torch.nn.modules.conv.Conv2d or
       type(layer) == torch.nn.modules.MaxPool2d):
        print( list( x.shape ) )
    print()

First Batch
torch.Size([7, 3, 150, 150])

### ENCODE ###
Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
[7, 6, 150, 150]

ReLU()

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
[7, 6, 75, 75]

Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
[7, 12, 75, 75]

ReLU()

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
[7, 12, 37, 37]

Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
[7, 24, 37, 37]

ReLU()

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
[7, 24, 18, 18]



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
